# ReaDDy intro II: Reactions, observables and checkpoints

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import readdy

In [ ]:
data_dir = "/enter/directory/here"
if not os.path.exists(data_dir):
    raise ValueError("HALT STOP")

## Simple observable example
`A + A -> B`, observe number of particles

In [ ]:
system = readdy.ReactionDiffusionSystem(box_size=[20.,20.,20.], unit_system=None)

In [ ]:
system.add_species("A", diffusion_constant=1.0)
system.add_species("B", diffusion_constant=1.0)

In [ ]:
system.reactions.add("myfusionname: A +(1) A -> B", rate=0.1)

In [ ]:
simulation = system.simulation()

In [ ]:
simulation.output_file = os.path.join(data_dir, "fusion.h5")
simulation.observe.number_of_particles(10, ["A", "B"], callback=lambda x: print(f"A {x[0]} B {x[1]}"))

In [ ]:
simulation.record_trajectory(10)

In [ ]:
simulation.add_particles("A", positions=np.random.uniform(size=(1000,3))*20. - 10.)

In [ ]:
if os.path.exists(simulation.output_file):
    os.remove(simulation.output_file)

simulation.run(10000, timestep=0.01)

In [ ]:
traj = readdy.Trajectory(simulation.output_file)

In [ ]:
traj.convert_to_xyz(particle_radii={"A": 0.5, "B": 0.5}, draw_box=True)

In [ ]:
times, numbers = traj.read_observable_number_of_particles()
print(times.shape)
print(numbers.shape)

In [ ]:
plt.plot(times, numbers[:,0])
plt.xlabel("Timestep")
plt.ylabel("Number of particles")

## Simple checkpointing example
Equilibrating a soft particle suspension

In [ ]:
crowded = False
name = "crowded" if crowded else "free"

n_particles = 1000
origin = np.array([-9.,-9.,-9.])
extent = np.array([18.,18.,18.])

data_dir = "/home/chris/workspace/data/workshop"
out_file = os.path.join(data_dir, f"{name}.h5")
checkpoint_dir = os.path.join(data_dir, f"ckpts_{name}")
n_steps = 20000
dt = 1e-2

In [ ]:
system = readdy.ReactionDiffusionSystem(
    [20.,20.,20.], 
    periodic_boundary_conditions=[False, False, False],
    unit_system=None)

system.add_species("A", 0.1)

system.potentials.add_box("A", 100., origin=origin, extent=extent)

if crowded:
    system.potentials.add_harmonic_repulsion("A", "A", force_constant=100., interaction_distance=2.)

In [ ]:
simulation = system.simulation("SingleCPU")
simulation.output_file = out_file

simulation.observe.particle_positions(stride=1)

if os.path.exists(checkpoint_dir):
    simulation.load_particles_from_latest_checkpoint(checkpoint_dir)
else:
    init_pos = np.random.uniform(size=(n_particles, 3)) * extent + origin
    simulation.add_particles("A", init_pos)

# this also creates the directory, if it does not exist
simulation.make_checkpoints(n_steps//100, output_directory=checkpoint_dir, max_n_saves=10)

if os.path.exists(simulation.output_file):
    os.remove(simulation.output_file)
    
simulation.run(n_steps, dt)